In [ ]:
names = ["train", "test", "valid"]

for name in names:
    # Python script to concatenate prompts and stories
    with open('data/hd/prepro/' + name + '.wp_source', 'r', encoding='utf-8') as sources, \
         open('data/hd/prepro/' + name + '.wp_target', 'r', encoding='utf-8') as targets, \
         open('data/hd/prepro/combined0/' + name + '_combined.txt', 'w', encoding='utf-8') as outfile:
        for prompt, story in zip(sources, targets):
            outfile.write(prompt.strip() + "\n" + story.strip() + "\n\n")


In [2]:
!deepspeed run_clm.py \
    --model_name_or_path distilgpt2 \
    --train_file data/hd/prepro/combined0/train_combined.txt \
    --validation_file data/hd/prepro/combined0/valid_combined.txt \
    --do_train \
    --do_eval \
    --output_dir ./models/distilgpt2-finetuned_gen0 \
    --num_train_epochs 5 \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --deepspeed ds_config.json \
    #--resume_from_checkpoint ./models/distilgpt2-finetuned_gen0/checkpoint-28500


[2024-04-12 15:30:06,030] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-12 15:30:06,618] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-04-12 15:30:06,630] [INFO] [runner.py:568:main] cmd = /home/vasi/Documents/BA_Thesis_Experiment/.venv/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMV19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None run_clm.py --model_name_or_path distilgpt2 --train_file data/hd/prepro/combined/train_combined.txt --validation_file data/hd/prepro/combined/valid_combined.txt --do_train --do_eval --output_dir ./models/distilgpt2-finetuned --num_train_epochs 5 --learning_rate 5e-5 --per_device_train_batch_size 4 --gradient_accumulation_steps 4 --deepspeed ds_config.json --resume_from_checkpoint ./models/distilgpt2-finetuned/checkpoint-28500
[2024-04-12 15:30:08,583] [INFO] [real_a

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "./models/distilgpt2-finetuned_gen0/checkpoint-31000" 
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


prompt = "Create a story about everlasting love. "  # replace with your own prompt
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


# Generate text
output_sequences = model.generate(
    input_ids=inputs,
    attention_mask=None,
    max_length=500,  # determines the maximum length of the generated text
    temperature=0.7,  # controls randomness: lower values make text less random
    top_k=50,  # the K most likely next words are considered for each step
    top_p=0.9,  # only the most probable tokens with probabilities that add up to top_p are considered for each step
    repetition_penalty=1.2,  # penalty applied to repeated words
    do_sample=True,  # set to True to return diverse samples
    num_return_sequences=1,  # number of independently computed samples to generate
    pad_token_id=tokenizer.eos_token_id,
)

# Decode the output sequences to get the generated text
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)


print(generated_text)


Create a story about everlasting love. ____________________________________________________________________________________________________________________ <newline> <newline> `` What?! '' she said. She was standing on the opposite side of the room, looking up at me from behind her. <newline> I stared at my reflection in the mirror for a moment before speaking. <newline> `` You're the one that hasn't spoken to anyone since you were seven. '' <newline>
[ WP ] A new technology is being developed which allows people to find and share information with their loved ones through video chat or by simply touching them without actually saying anything... until they meet someone who can no longer communicate verbally/physically ( if it means physically ). One day while searching your phone number there appears to be an odd message : `` My name is Lucy. Please select this line as your personal text only ; not mine. ''
“ Are you sure we ’ re doing alright? ” <newline> <newline> “ Sure, ” John repli

In [6]:
with open("./outputs/gen0/love_story0.txt", 'w') as f:
    f.write(generated_text)

In [17]:
from metrics.LexicalDiversity.lexical_diversity import *

with open("./outputs/gen0/love_story0.txt", 'r') as f:
    story = f.read()

print("GEN0")
print("Lexical Diversity: ")
print("Distinct n: ", calculate_distinct_n(story, 2))
print(f"Self_BLEU Score: {1-calculate_self_bleu(story)}")
print("TTR: ", calculate_ttr(story, 300))

GEN0
Lexical Diversity: 
Total n_grams:  339
Unique n_grams:  310
Distinct n:  0.9144542772861357
Self_BLEU Score: 1.0
Total Tokens:  340
TTR:  0.72
